<h3>TensorFlow Deep Learning

In [1]:
import tensorflow as tf
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<h3>Create Model

In [2]:
from collections import OrderedDict

參考資料：<a href="http://www.zlovezl.cn/articles/collections-in-python/" target="_blank">不可不知的Python模块: collections</a>

In [28]:
def create_dense_layer(inputs, n_nodes, activation=None, dropout=None):
    return tf.layers.dense(inputs=inputs, units=n_nodes, activation=activation)

def create_dropout(inputs, rate, mode):
    return tf.layers.dropout(inputs=inputs, rate=rate, training=mode == tf.estimator.ModeKeys.TRAIN)

## tf.logging.set_verbosity(tf.logging.INFO)

def deep_model_fn(features, labels, mode, params):
    ### 深度學習模型的函數
    
    layers_dict = OrderedDict()
    ## 輸入層
    layers_dict["input_layer"] = features["x"]
    
    ## 依照layer的參數 創造 layers
    for i, layer in enumerate(params["layers"]):
        layer["inputs"] = list(layers_dict.values())[-1]
        if layer == params["layers"][-1]:  ## 判斷假如已經到了最後一層layer，那麼就是輸出層
            key = "logits"
        else:
            key = "layer" + str(i)
        layers_dict[key] = create_dense_layer(**layer)
        if layer["dropout"]:
            layers_dict["dropout_l"+str(i)] = create_dropout(list(layers_dict.values())[-1], rate=layer["dropout"], mode=mode)
      
    print(list(layers_dict.keys()))
    
    predictions = {
        ## 產生預測結果(for PREDICT and EVAL 模式)
        "classes": tf.argmax(input=layers_dict["logits"], axis=1),
        # 在圖中加入"softmax_tensor"，用來給預測模式以及logging hook
        "probabilities": tf.nn.softmax(layers_dict["logits"], name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    ## 計算 LOSS (for both TRAIN and EVAL 模式)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=layers_dict["logits"])
    
    ## 架構訓練優化器(for TRAIN 模式)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=params["learning_rate"])
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
   
    ## 評估模式的評估指標(for EVAL 模式)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

<h3>Main Function

In [29]:
def main(model_params=None, mode="train", model_dir=None):
    ## 載入訓練資料以及評估資料
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    
    # Train
    train_data = mnist.train.images  ## Return np.array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    
    # Test
    eval_data  = mnist.test.images
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
    ## 創造評估器
    if not model_dir:
        import os
        model_dir = "/tmp/mnist_deep_model"
        if not(os.path.isdir(model_dir)):
            os.makedirs(model_dir)
        if mode == "train":
            os.system("rm -rf " + model_dir)
    mnist_classifier = tf.estimator.Estimator(model_fn=deep_model_fn, params=model_params, model_dir=model_dir)
    
    ## 訓練模型
    if mode == "train":
        train_input_fn = tf.estimator.inputs.numpy_input_fn(
                        x={"x": train_data},
                        y=train_labels,
                        batch_size=100,
                        num_epochs=10,
                        shuffle=True)
        mnist_classifier.train(input_fn=train_input_fn)
   
    ## 預測模型
    elif mode == "predict":
        predict_input_fn = tf.estimator.inputs.numpy_input_fn(
                        x={"x": eval_data},
                        num_epochs=1,
                        shuffle=False)
        preds = mnist_classifier.predict(input_fn=predict_input_fn)
        return [p for p in preds], eval_labels
    
    ## 評估模型 
    elif mode == "eval":
        eval_input_fn = tf.estimator.inputs.numpy_input_fn(
                        x={"x": eval_data},
                        y=eval_labels,
                        num_epochs=1,
                        shuffle=False)
        eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
        print(eval_results)

<h3>架構神經層

In [30]:
layers = [{"n_nodes": 1024, "activation": tf.nn.relu, "dropout": None},
          {"n_nodes": 512, "activation": tf.nn.relu, "dropout": 0.4},
          {"n_nodes": 256, "activation": tf.nn.relu, "dropout": None},
          {"n_nodes": 128, "activation": tf.nn.relu, "dropout": 0.4},
          {"n_nodes": 10, "activation": None, "dropout": None} ]

<h3>模型參數

In [31]:
model_params = {"learning_rate": 0.01,
                "layers": layers}

<h3>執行

In [32]:
main(model_params=model_params, mode="train")

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_deep_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002021EF69780>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
['input_layer', 'layer0', 'layer1', 'dropout_l1', 'layer2', 'layer3', 'dropout_l3', 'logits']
INFO:tens

<h3>評估預測

In [33]:
from sklearn.metrics import classification_report as crep, accuracy_score as acc

In [34]:
preds = main(model_params, mode="predict")

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_deep_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000020254C05780>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
['input_layer', 'layer0', 'layer1', 'dropout_l1', 'layer2', 'layer3', 'dropout_l3', 'logits']
INFO:tens

In [35]:
ypred = [c["classes"] for c in preds[0]]
ytrue = preds[1]

In [36]:
print("acc :", acc(ytrue, ypred))

acc : 0.9591
